# RAG with Agents

In this sample we will be creating and agent that is going to use a knowledge document to respond to user questions.

You should have the following two environment variables already created and initialized:

* `AZURE_AI_PROJECT_ENDPOINT`: Your Azure AI project endpoint
* `AZURE_AI_MODEL_DEPLOYMENT_NAME`: The name of your model deployment

In [ ]:
#import asyncio
import os
#from random import randint
from agent_framework import AgentRunResponse,ChatAgent,HostedFileSearchTool,HostedVectorStoreContent
from agent_framework.azure import AzureAIAgentClient
from azure.identity.aio import AzureCliCredential

project_endpoint = os.environ.get('AZURE_AI_PROJECT_ENDPOINT')
model_name = os.environ.get('AZURE_AI_MODEL_DEPLOYMENT_NAME')

print(f"Project endpoint: {project_endpoint}")
print(f"Deployment name: {model_name}")

In [ ]:
async def create_vector_store(client: AzureAIAgentClient) -> tuple[str, HostedVectorStoreContent]:
    """Create a vector store with sample documents."""
    file_path = './knowledge.md'
    file = await client.project_client.agents.files.upload_and_poll(file_path=file_path, purpose="assistants")
    print(f"Uploaded file, file ID: {file.id}")


    vector_store = await client.project_client.agents.vector_stores.create_and_poll(file_ids=[file.id], name="graph_knowledge_base")

    print(f"Created vector store, ID: {vector_store.id}")


    return file.id, HostedVectorStoreContent(vector_store_id=vector_store.id)

In [ ]:
async with (
        AzureCliCredential() as credential,
        AzureAIAgentClient(async_credential=credential) as chat_client,
    ):
        file_id, vector_store = await create_vector_store(chat_client)

        file_search = HostedFileSearchTool(inputs=vector_store)
        
        agent = chat_client.create_agent(
            name="PythonRAGDemo",
            instructions="""
                You are an AI assistant designed to answer user questions using only the information retrieved from the provided document(s).

                - If a user's question cannot be answered using the retrieved context, **you must clearly respond**: 
                "I'm sorry, but the uploaded document does not contain the necessary information to answer that question."
                - Do not answer from general knowledge or reasoning. Do not make assumptions or generate hypothetical explanations.
                - Do not provide definitions, tutorials, or commentary that is not explicitly grounded in the content of the uploaded file(s).
                - If a user asks a question like "What is a Neural Network?", and this is not discussed in the uploaded document, respond as instructed above.
                - For questions that do have relevant content in the document (e.g., Contoso's travel insurance coverage), respond accurately, and cite the document explicitly.

                You must behave as if you have no external knowledge beyond what is retrieved from the uploaded document.
                """,
            tools=[file_search],  # Tools available to the agent
            tool_choice = "auto",  # Let the agent decide when to use tools
        )
                

        print("Agent created. You can now ask questions about the uploaded document.")

        query = "Can you explain Contoso's travel insurance coverage?"
        async for chunk in agent.run_stream(query, tool_resources={"file_search": {"vector_store_ids": [vector_store.vector_store_id]}}):
                
            if chunk.text:
                print(chunk.text, end="", flush=True)
    